In [ ]:
# Import all the necessary library
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

# EXTRACT

In [ ]:
# Extract the two dataset CSVs into DataFrames
## Load Happiness data
happiness_file = 'Resources/world_happiness_2019.csv'
happiness_df = pd.read_csv(happiness_file)
happiness_df.head(3)

In [ ]:
countries_file = "Resources/countries of the world.csv"
countries_df = pd.read_csv(countries_file)
countries_df.head(3)

# TRANSFORM

We performed data transformation and clean-up on our two dataset in order for them to work properly.

For each of the dataset, we transformed our data with the following:
    1. Selected only the columns of our interest within our two dataset.
    2. Created copies of the two dataset with the new columns we selected.
    3. Renamed the columns to be simpler to type with no spaces.
 
Once the clean-up is complete, we cleaned-up both our dataset with the following:
    1. Removed any NaN values from our dataset, if there's any.
    2. Removed any duplicative rows with the same 'country'.
    3. Set the index to be 'country' column.

### Wolrd Happiness

In [ ]:
# Create a filtered dataframe from specific columns
happiness_df_col = ['Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']
happiness_transformed_df = happiness_df[happiness_df_col].copy()

# Rename the column headers
happiness_transformed_df = happiness_transformed_df.rename(columns={
    'Country or region' : 'country',
    'Score': 'overall_score', 
    'GDP per capita': 'gdp_score',
    'Social support': 'social_support', 
    'Healthy life expectancy': 'healthy_life_expectancy', 
    'Freedom to make life choices': 'freedom_choices',
    'Generosity': 'generosity', 
    'Perceptions of corruption': 'corruption_perception'
})

# Clean the data by dropping duplicates and setting the index
happiness_transformed_df = happiness_transformed_df.dropna(how='any')
happiness_transformed_df.drop_duplicates('country', inplace=True)
happiness_transformed_df.set_index('country', inplace=True)
happiness_transformed_df

### Countries

A few of the columns in our Countries dataset are in the string format, and the data in 'country' column has an extra space after the text. To ensure our two datasets load and join properly, we performed additional data-clean up for our Countries dataset prior to the clean-up step outlined previously:
    1. Changed the data types of the following columns from string to float: pop_density, infant_mortality, literacy;
    2. Removed the extra space in the 'country' column.

In [ ]:
# Create a filtered dataframe from specific columns
countries_df_col = ['Country', 'Region', 'Pop. Density (per sq. mi.)', 'Infant mortality (per 1000 births)', 
                    'GDP ($ per capita)', 'Literacy (%)']
countries_transformed_df = countries_df[countries_df_col].copy()

# Rename the column headers
countries_transformed_df = countries_transformed_df.rename(columns={
    'Country': 'country', 
    'Region': 'region', 
    'Pop. Density (per sq. mi.)': 'pop_density', 
    'Infant mortality (per 1000 births)': 'infant_mortality', 
    'GDP ($ per capita)': 'gdp', 
    'Literacy (%)': 'literacy'
})

## Extra Clean-up
# Convert columns with string of numbers to float
countries_transformed_df['pop_density'] = countries_transformed_df.pop_density.str.replace(',', '.').astype(float)
countries_transformed_df['infant_mortality'] = countries_transformed_df.infant_mortality.str.replace(',', '.').astype(float)
countries_transformed_df['literacy'] = countries_transformed_df.literacy.str.replace(',', '.').astype(float)

# Remove the weird space after text in country column
countries_transformed_df['country'] = countries_transformed_df['country'].str.strip()

## Default Clean-up step
# Clean the data by dropping NaN, duplicates and setting the index
countries_transformed_df = countries_transformed_df.dropna(how='any')
countries_transformed_df.drop_duplicates('country', inplace=True)
countries_transformed_df.set_index('country', inplace=True)
countries_transformed_df

### Checking to make sure the two tables merge

We confirmed that our two datasets can merge properly at the index.

In [ ]:
# Verify that two datasets can merge properly
merge_df = pd.merge(happiness_transformed_df, countries_transformed_df, on='country')
merge_df

Once the transformation step is complete, we set up our database connection with postgreSQL. The {username} and {password} will be pull from config.py. Our database of interest in postgreSQL is 'countries_happiness_db'.

Then, we confirmed that the two tables created in postgreSQL get connected to python.

In [ ]:
# Create database connection
connection_string = f'{username}:{password}@localhost:5432/countries_happiness_db'
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

# LOAD

In [ ]:
# Load Happiness Df into database
happiness_transformed_df.to_sql(name='happiness', con=engine, if_exists='append', index=True)

In [ ]:
# Load Countries Df into database
countries_transformed_df.to_sql(name='country', con=engine, if_exists='append', index=True)